In [1]:
import pandas as pd

In [2]:
messages = pd.read_csv('spam.csv',encoding='latin1')

In [3]:
messages

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [5]:
messages.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
messages.rename(columns={'v1': 'label', 'v2': 'message'}, inplace=True)

print(messages.head())

  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [6]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
messages.shape

(5572, 2)

In [8]:
messages['message'].loc[99]

"Please don't text me anymore. I have nothing else to say."

## Data Cleaning and Pre-processing

## Using Machine Learning
### Text-Preprocessing (Tokenization->Stopwords->skemming and Lemmatization, Nltk)
### Text into Vectprs (BOW,TFIDF, Word2vec, AvgWrod2Vec)

In [9]:
import re 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/wahid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [11]:
corpus = []

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z0-9]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

corpus

### Creating the Bag of words model

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500,binary=True,ngram_range=(2,2))
X = cv.fit_transform(corpus).toarray()

In [13]:
X.shape

(5572, 2500)

In [14]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
y = pd.get_dummies(messages['label'])
y =y.iloc[:,1].values

In [16]:
y

array([False, False,  True, ..., False, False, False])

### Train Test Split

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [18]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [19]:
y_pred = spam_detect_model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score, classification_report

In [21]:
score = accuracy_score(y_test, y_pred)
print(score)

0.9623318385650225


In [22]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       1.00      0.96      0.98       989
        True       0.75      0.99      0.86       126

    accuracy                           0.96      1115
   macro avg       0.88      0.98      0.92      1115
weighted avg       0.97      0.96      0.96      1115



### Using TFIDF Instead of BOW

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=2500,binary=True,ngram_range=(2,2))
out_tf = tf.fit(corpus)
X_Tf = tf.fit_transform(corpus).toarray()

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_Tf, y, test_size=0.20, random_state=0)

### Model Ml DL

In [25]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)
y_pred = spam_detect_model.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score, classification_report

In [27]:
score = accuracy_score(y_test, y_pred)
print(score)

0.9461883408071748


In [28]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       1.00      0.94      0.97      1007
        True       0.64      0.99      0.78       108

    accuracy                           0.95      1115
   macro avg       0.82      0.97      0.88      1115
weighted avg       0.96      0.95      0.95      1115



In [29]:
import pickle

In [32]:
# Save the model and TF-IDF Vectorizer using pickle
with open('spam_model.pkl', 'wb') as f:
    pickle.dump(spam_detect_model, f)

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(out_tf, f)